<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [57]:
import pandas as pd
import dill
import sys
sys.path.insert(0, '..\\NOTEBOOKS')
from P7_functions import CustTransformer

In [58]:
with open('data\\bestmodel_thresh.pkl', 'rb') as file:
    best_model, thresh = dill.load(file)

In [59]:
# with open('data\\dict_cleaned.pkl', 'rb') as file:
#     dict_cleaned_all = dill.load(file)
# dict_cleaned_all.keys()

In [60]:
# dict_cleaned_all['X_train'].shape, dict_cleaned_all['y_train'].shape, dict_cleaned_all['X_test'].shape

In [61]:
# ind_samp_tr = dict_cleaned_all['X_train'].sample(20000).index
# ind_samp_te = dict_cleaned_all['X_test'].sample(5000).index
# X_train_samp = dict_cleaned_all['X_train'].loc[ind_samp_tr]
# y_train_samp = dict_cleaned_all['y_train'].loc[ind_samp_tr]
# X_test_samp = dict_cleaned_all['X_test'].loc[ind_samp_te]

In [62]:
# dict_cleaned = {}
# dict_cleaned['X_train'] = X_train_samp
# dict_cleaned['y_train'] = y_train_samp
# dict_cleaned['X_test'] = X_test_samp

# with open('data\\dict_cleaned_samp.pkl', 'wb') as file:
#     dill.dump(dict_cleaned, file)

In [ ]:
# dict_cleaned['X_train'].shape, dict_cleaned['y_train'].shape, dict_cleaned['X_test'].shape

In [69]:
with open('data\\dict_cleaned_samp.pkl', 'rb') as file:
    dict_cleaned = dill.load(file)
# best model and best threshold
with open('data\\bestmodel_thresh.pkl', 'rb') as file:
    best_model, thresh = dill.load(file)

# compute processed data (first steps of the best_model pipeline)
X_train = dict_cleaned['X_train']
y_train = dict_cleaned['y_train']
X_test = dict_cleaned['X_test']

In [74]:
X_test.index.sort_values()

Int64Index([100128, 100169, 100278, 100306, 100312, 100438, 100484, 100494,
            100510, 100517,
            ...
            455481, 455489, 455513, 455537, 455648, 455742, 456010, 456169,
            456189, 456202],
           dtype='int64', name='SK_ID_CURR', length=5000)

In [70]:
# compute processed data (first steps of the best_model pipeline)
X_train = dict_cleaned['X_train']
y_train = dict_cleaned['y_train']
X_test = dict_cleaned['X_test']
# split the steps of the best pipeline
preproc_step = best_model.named_steps['preproc']
featsel_step = best_model.named_steps['featsel']
clf_step = best_model.named_steps['clf']
# compute the preprocessed data (encoding and standardization)
X_tr_prepro = preproc_step.transform(X_train)
X_te_prepro = preproc_step.transform(X_test)
# get the name of the columns after encoding
preproc_cols = X_tr_prepro.columns
# get the name of the columns selected using SelectFromModel
featsel_cols = preproc_cols[featsel_step.get_support()]
# compute the data to be used by the best classifier
X_tr_featsel = X_tr_prepro[featsel_cols]
X_te_featsel = X_te_prepro[featsel_cols]

In [71]:
X_tr_featsel.index.sort_values()

Int64Index([100007, 100009, 100010, 100012, 100022, 100029, 100040, 100041,
            100086, 100130,
            ...
            456094, 456099, 456125, 456129, 456143, 456147, 456165, 456187,
            456191, 456236],
           dtype='int64', name='SK_ID_CURR', length=20000)

In [72]:
from flask import Flask, jsonify, request
import json
from sklearn.neighbors import NearestNeighbors

# Parse the http request to get arguments (sk_id_cust)
# sk_id_cust = int(request.args.get('SK_ID_CURR'))
sk_id_cust = 100009
# get data of 20 nearest neigh in the X_tr_featsel dataframe (pd.DataFrame)
neigh = NearestNeighbors(n_neighbors=20)
neigh.fit(X_tr_featsel)
idx = neigh.kneighbors(X=X_tr_featsel.loc[sk_id_cust: sk_id_cust],
                       n_neighbors=20,
                       return_distance=False).ravel()
nearest_cust_idx = list(X_tr_featsel.iloc[idx].index)
X_neigh_df = X_tr_featsel.loc[nearest_cust_idx, :]
# Convert the pd.DataFrame (20 df rows) of customer's data to JSON
X_neigh_json = json.loads(X_neigh_df.to_json())
# Return the cleaned data jsonified
# jsonify({'status': 'ok',
#          'data': X_neigh_json})

In [76]:
X_neigh_df.mean()

binary__CODE_GENDER                            1.050000
high_card__OCCUPATION_TYPE                     0.067642
high_card__ORGANIZATION_TYPE                   0.076803
Bur_CB_AMT_CREDIT_MAX_OVERDUE__max            -0.005782
PA_PrLCash_SK_DPD_DEF__mean__mean             -0.135742
PA_DAYS_FIRST_DUE__min                        -0.067499
Bur_CB_AMT_CREDIT_SUM_LIMIT__sum              -0.211547
PA_DAYS_LAST_DUE_1ST_VERSION__min             -0.091919
PA_PrLI_DELAY_DAYS_INSTALMENT__max__max       -0.063639
Bur_CB_DAYS_CREDIT_ENDDATE__max               -0.263093
DAYS_EMPLOYED_PERC                             0.479074
PA_DAYS_FIRST_DUE__max                        -0.285695
Bur_CB_AMT_CREDIT_SUM_DEBT__sum                0.235727
PA_PrLI_NUM_INSTALMENT_NUMBER__max__max       -0.110137
Bur_CB_AMT_CREDIT_SUM__mean                   -0.003855
DAYS_LAST_PHONE_CHANGE                        -0.599322
PA_PrLI_DELAY_DAYS_INSTALMENT__mean__mean      0.219180
PA_PrLCash_MONTHS_BALANCE__abs_min__abs_min   -0